In [9]:
# Create biomet and dynamicMetadata.csv files
import os
import sys
import importlib

BiometNet = 'C:/Biomet.net/Python/'

# UBC Micromet users can 
if sys.path[0]!=BiometNet:
    sys.path.insert(0,BiometNet)

wd = [p for p in sys.path if p != BiometNet][0]

import csvFromBinary as cfb
importlib.reload(cfb)

siteID = 'BBS'
dateRange = ['2023-06-01','2024-05-31']
tasks=os.path.abspath(wd+'/config_files/EP_auxillary_data_defs.yml')
auxilaryDpaths=cfb.makeCSV(siteID,dateRange,tasks,
                           outputPath=f'C:/highfreq/{siteID}/auxilaryData',stage='Second')
print(auxilaryDpaths)

Initializing tasks for BBS over: ['2023-06-01', '2024-05-31']
PPFD_IN_1_1_1 missing, outputting NaNs
See output: C:/highfreq/BBS/auxilaryData/BBS_biometData_202306010000_202405310000.csv
See output: C:/highfreq/BBS/auxilaryData/BBS_dynamicMetadata_202306010000_202405310000.csv
All tasks completed successfully
{'biometData': 'C:/highfreq/BBS/auxilaryData/BBS_biometData_202306010000_202405310000.csv', 'dynamicMetadata': 'C:/highfreq/BBS/auxilaryData/BBS_dynamicMetadata_202306010000_202405310000.csv'}


In [7]:
import eddyProAPI
import pandas as pd
import importlib
import time
importlib.reload(eddyProAPI)

# Basic call for .ghg files

siteID = 'BBS'
dateRange=[(pd.Timestamp.now()-pd.Timedelta(days=365)).strftime('%Y%m%d'),pd.Timestamp.now().strftime('%Y%m%d')],


if siteID == 'BB':
    pp = eddyProAPI.preProcessing('BB',dateRange=dateRange,Testing=5)
    pp.searchRawDir()
elif siteID == 'BBS':
                
    # # Basic call for .dat files
    metaDataTemplate='C:/highfreq/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata'
    pp = eddyProAPI.preProcessing('BBS',dateRange=dateRange,fileType='dat',metaDataTemplate=metaDataTemplate,
                                # debug=True,
                                # reset=True,
    )
    pp.searchRawDir(
                copyFrom = "C:/Campbellsci/CardConvert",
                searchTag = "BBS.FLUX",
                timeShift=30)
pp.readFiles()
pp.groupAndFilter()
# with shutil 186.0170259475708 seconds
# via subprocess

Searching C:/highfreq//BBS/raw/2024\03
[████████████████████████████████████████████████████████████] 1169/1169

Searching C:/highfreq//BBS/raw/2024\04
[████████████████████████████████████████████████████████████] 1440/1440

Searching C:/highfreq//BBS/raw/2024\05
[████████████████████████████████████████████████████████████] 462/462

Searching C:/Campbellsci/CardConvert
[████████████████████████████████████████████████████████████] 3/3

Searching C:/Campbellsci/CardConvert\20240204
[████████████████████████████████████████████████████████████] 54/54

Searching C:/Campbellsci/CardConvert\20240213
[████████████████████████████████████████████████████████████] 4/4

Searching C:/Campbellsci/CardConvert\20240223
[████████████████████████████████████████████████████████████] 911/911

Searching C:/Campbellsci/CardConvert\20240307
[████████████████████████████████████████████████████████████] 627/627

Searching C:/Campbellsci/CardConvert\20240416
[█████████████████████████████████████████████

In [50]:
# Cospectral correction options
hf_meth = {
    'Moncrieff et al. (1997)':1,
    'Horst (1997)':2,
    'Ibrom et al. (2007)':3,
    'Fratini et al. (2012)':4, # Recommended , when sufficient data (>1 month) are available
    'Massman (2000, 2001)':5
}

userDefinedEddyProSettings = {
    'Project':{
        'hf_meth':hf_meth['Fratini et al. (2012)'],
        },
    'RawProcess_Settings':{
        'v_offset':0.08,
        },
    }

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)

siteID = 'BBS'


eP = eddyProAPI.runEP(siteID,userDefinedEddyProSettings=userDefinedEddyProSettings,processes=4,**auxilaryDpaths)
# eP.configurationGroups

Creating C:/highfreq//BBS/metadata/group_1_202402031330_202402290700.eddypro
Creating C:/highfreq//BBS/metadata/group_1_202402290700_202403260000.eddypro
Creating C:/highfreq//BBS/metadata/group_1_202403260000_202404201700.eddypro
Creating C:/highfreq//BBS/metadata/group_1_202404201700_202405161000.eddypro


In [18]:
# if 'Project' in userDefinedEddyProSettings.keys():
#     print('P')
eP.groupMetaData['Instruments']['instr_1_model']

'csat3_1'

In [ ]:
## To DO: Add Output ID on merge so we can trace down specific thread/log file for a given run
## Same timestamp can have different outputs depending on batch it was run in
## Usually minor differences, but can be larger in select circumstances

import setupEP as eP
import importlib
import time
importlib.reload(eP)

########## Note - incomplete ghg or biomet files can crash program (preprocessing procedures should help prevent that by re-naming incomplete files)

T1 = time.time()
mR = eP.makeRun('ep_Templates/DefaultSettings.eddypro','BBS',Processes=6,priority = 'high priority')
mR.runDates(['2023-01-01 00:00','2024-03-31 23:59'])

# mR = eP.makeRun('ep_Templates/LabStandard_Advanced.eddypro','BB',Processes=4,priority = 'high priority')
# mR.updateTemplate({'FluxCorrection_SpectralAnalysis_General':
#                     {'sa_mode':4}
#                 })
# mR.runDates(['2015-01-01 00:00','2023-12-31 23:59'])

T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60)

## Read .ghg metadata

In [ ]:
import preProcessing
import importlib
import time
importlib.reload(preProcessing)
T1 = time.time()

# Need one or more metadata templates if working with .dat files
template = ['Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata']

copyFrom = "X:/BBS/EC_Station/2023/20230616"
copyTag = "BBS.FLUX"
pr = preProcessing.read_ALL('BBS',
                            fileType='dat',
                            reset=1,
                            metadataTemplate=template,
                            copyFrom=copyFrom,
                            copyTag=copyTag,
                            timeShift=30)
# pr.find_files(2023,6,processes=4)

# updates = 'Y:/BB/Metadata_Updates.csv'
# pr = preProcessing.read_ALL('BB',fileType='ghg',metadataUpdates=updates,reset=1)
# # If updates/corrections must be made to metadata (e.g., undocumented orientation changes) a .csv files of updates can be provided

# for y in range(2015,2024):
#     for m in range(1,13):
#         pr.find_files(y,m)

# T2 = time.time()

# print('\n')
# print('Runtime: ',(T2-T1)/60,' minutes')


In [ ]:
# ## Implement pre-processing procedures to exclude data by conditions (e.g., low flow rate)
pr.ini['dat']
# import numpy as np
# pr.dataRecords['flow_rate_7200']
# pr.dataRecords.loc[pr.dataRecords['flow_rate_7200'].abs()>1e2,'flow_rate_7200']=np.nan
# pr.dataRecords['flow_rate_7200'].plot()
# # pr.dataRecords['col_air_t'].plot()#['altitude'].describe()
# 317/60

In [ ]:
# https://www.licor.com/env/support/EddyPro/topics/low-pass-filtering.html

# bin_sp_avail=[0,1]
# full_sp_avail=[0,1]
# sa_bin_spectra='Path'
# sa_full_spectra = 'Path'

## Full Runtime

### Reading & Writing all to Y: drive:

1 Month of BB data (20 hz)

**Preprocessing**: 3.91 minutes (8 cores)
**Processing**: 56.92 minutes (6 cores)

Total: **60.83** minutes

### Reading & Preprocessing on Y: drive then writing EddyPro results locally:

1 Month of BB data (20 hz)

**Preprocessing**: 3.91 minutes (8 cores)
**Processing**: 55.74 minutes (6 cores)

Total: **59.65** minutes

### Copying data to C then writing EddyPro results locally:

1 Month of BB data (20 hz)

**Preprocessing**: 6.30 minutes (1 core copy > 8 core preprocessing)

**Processing**: 54.33 minutes (6 cores)

Total: **60.3** minutes

* Paralellizing data copy could shave enought time to get marginal benfit, but doesn't look to matter much

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.read_csv(mR.all_outputs['fulloutput'],skiprows=[0,2],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')
# Add routine to handle duplicates (or find rood cause of them )
# Occuring on boundary between batches I assume as there are ~ 2x as many duplicates as batches
df = df.loc[df.index.duplicated()==False].copy()

bm = pd.read_csv(mR.all_outputs['biomet'],skiprows=[1],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')
bm = bm.loc[bm.index.duplicated()==False].copy()


df = pd.concat([df,bm[bm.columns[~bm.columns.isin(df.columns)]]],axis=1)
df.loc[df['qc_co2_flux']>0,'co2_flux']=np.nan
df.loc[df['u*']<0.15,'co2_flux']=np.nan

plt.scatter(df['Rg_1_1_1'],df['co2_flux'])

# df.loc[df['co2_flux']>20,['wind_speed','u*','wind_dir','Ta_1_1_1','H']]


In [ ]:
import csvFromBinary as cfb
import importlib
importlib.reload(cfb)


# BM
cfb.makeBiomet('BBS',['2023-06-01 00:00','2024-05-31 23:59'],'C:/highfreq/BBS/metadata',type='dynamicMetadata')


In [ ]:
# t = ''
# type(t)

In [ ]:
# df.rename(columns=lambda c: d[c].pop(0) if c in d.keys() else c)
# bm.duplicated().sum()
# bm.columns[bm.columns.duplicated()]
# T1 = df.iloc[0:10].copy()
# T2 = bm[bm.columns[~bm.columns.isin(df.columns)]].iloc[0:10].copy()


# T3 = pd.concat([T1,T2],axis=1)
# T3
rep = df[df.index.duplicated(keep=False)].sort_index().copy()#.duplicated()#['co2_flux']
# print(rep.shape)
for c in rep.columns:
    s = rep.loc[rep[c].duplicated(keep=False)==False,c]

    if s.shape[0]>1:
        # print(c)
        # print(s.shape[0])
        print(rep[['filename',c]].groupby('filename').diff().dropna().min())
        print()

In [ ]:
rep.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
monc = 'Y:/BB/Spectral_Comp/Moncrieff/'+'eddypro_BB_20230601_20230630_full_output_2024-03-15T153700_adv.csv'
frat = 'Y:/BB/Spectral_Comp/Fratini/'+'eddypro_Spectral_Correction_fratini_full_output_2024-03-19T000447_adv.csv'
df_fratini = pd.read_csv(frat,skiprows=[0,2],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')
df_moncrieff = pd.read_csv(monc,skiprows=[0,2],na_values=-9999,parse_dates={'TIMESTAMP':['date','time']},index_col='TIMESTAMP')

def Filt(df):
    df['filt'] = 1
    df.loc[df['u*']<.15,'filt']=np.nan
    df.loc[df['qc_co2_flux']>0,'filt']=np.nan
    df.loc[df['qc_LE']>0,'filt']=np.nan
    return(df)

df_fratini = Filt(df_fratini)
df_moncrieff = Filt(df_moncrieff)


# df_fratini=df_fratini.loc[df_fratini.index.month==6].copy()

cols = ['filt','u*','co2_flux','qc_co2_flux','LE','qc_LE','H','qc_H','ch4_flux','qc_ch4_flux']
frat_cols = {key: key+'_fratini' for key in cols}
monc_cols = {key: key+'_moncrieff' for key in cols}

df_fratini = df_fratini[cols].rename(columns=frat_cols)
df_moncrieff = df_moncrieff[cols].rename(columns=monc_cols)

df = df_fratini.join(df_moncrieff)

df['filt']=df['filt_fratini']*df['filt_moncrieff']

for col in df.columns:
    if col !='filt':
        df[col]=df[col]*df['filt']


fig,ax=plt.subplots(2,2,figsize=(8,8))


ax[0,0].scatter(df['co2_flux_fratini'],df['co2_flux_moncrieff'],label=f"r2 = {(df[['co2_flux_fratini','co2_flux_moncrieff']].corr()**2)['co2_flux_moncrieff'].round(3).values[0]}")
ax[0,0].set_xlabel('Fratini et al. (2012)')
ax[0,0].set_ylabel('Moncrieff et al. (1997)')
ax[0,0].set_title('CO2 Flux (umol m-2 s-1)')
ax[0,0].legend()
ax[0,0].grid()

df[['co2_flux_fratini','co2_flux_moncrieff']].boxplot(ax=ax[0,1])
ax[0,1].scatter([1,2],df[['co2_flux_fratini','co2_flux_moncrieff']].mean().values,marker='*',s=40,color='k')
ax[0,1].set_xticklabels(['Fratini et al. (2012)','Moncrieff et al. (1997)'])
ax[0,1].set_title('CO2 Flux (umol m-2 s-1)')


ax[1,0].scatter(df['LE_fratini'],df['LE_moncrieff'],label=f"r2 = {(df[['LE_fratini','LE_moncrieff']].corr()**2)['LE_moncrieff'].round(3).values[0]}")
ax[1,0].set_xlabel('Fratini et al. (2012)')
ax[1,0].set_ylabel('Moncrieff et al. (1997)')
ax[1,0].set_title('Latent Heat Flux (W m-2)')
ax[1,0].legend()
ax[1,0].grid()

df[['LE_fratini','LE_moncrieff']].boxplot(ax=ax[1,1])
ax[1,1].scatter([1,2],df[['LE_fratini','LE_moncrieff']].mean().values,marker='*',s=40,color='k')
ax[1,1].set_xticklabels(['Fratini et al. (2012)','Moncrieff et al. (1997)'])
ax[1,1].set_title('Latent Heat Flux (W m-2)')

plt.tight_layout()


df[['LE_fratini','LE_moncrieff','co2_flux_fratini','co2_flux_moncrieff','ch4_flux_fratini','ch4_flux_moncrieff',]].describe().round(3)

# Speedtests

All times in minutes

### 4 threads (8 timesteps each)

Normal priority: 2.05
High priority: 1.91

### 4 threads (48 timesteps each x 12 days)

High priority: 29

### Eddypro GUI (1 run [12 days, 576 timesteps])

High priority: 90